In [1]:
def remove_quotation(x):
    return([xx.replace('"', '') for xx in x])
def save_package_count(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(\
        lambda row: Row(time=time, package=row[0], count=row[1])))
        df.write.format("org.apache.spark.sql.cassandra")\
        .options(table="package_count", keyspace="streaming")\
        .save(mode="append")
    except:
        pass
def save_top_package(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(\
        lambda row: Row(time=time, package=row[0], count=row[1])))
        df = df.limit(1)
        df.write.format("org.apache.spark.sql.cassandra")\
        .options(table="top_package", keyspace="streaming")\
        .save(mode="append")
    except:
        pass
def save_country(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(\
        lambda row: Row(time=time, country=row[0], count=row[1])))
        df = df.limit(5)
        df.write.format("org.apache.spark.sql.cassandra")\
        .options(table="top_country", keyspace="streaming")\
        .save(mode="append")
    except:
        pass
def save_ggplot2_count(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(\
        lambda row: Row(time=time, ggplot2=row[0], count=row[1])))
        df.write.format("org.apache.spark.sql.cassandra")\
        .options(table="ggplot2_count", keyspace="streaming")\
        .save(mode="append")
    except:
        pass

In [2]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
import time
ssc = StreamingContext(sc, 3)
lines = ssc.textFileStream("file:///home/jawad/streaming/")
downloads_RDD = lines.map(lambda x: x.split(','))

downloads_RDD = downloads_RDD.map(remove_quotation)
# 1 - Package Count
package_RDD = downloads_RDD.map(lambda x: (x[6], 1))
package_RDD = package_RDD.reduceByKey(lambda a,b: a+b)

# 2 - highest downloaded package
package_RDD_reduced = package_RDD.transform(lambda package_RDD: package_RDD.sortBy(lambda a: a[1], ascending=False))

# 3 - Top 5 countries with number of downlaods
country_RDD = downloads_RDD.map(lambda x: (x[8], 1))
country_RDD = country_RDD.reduceByKey(lambda a,b: a+b)
country_RDD = country_RDD.transform(lambda downloads_RDD_reduced: downloads_RDD_reduced.sortBy(lambda x: x[1], 
                                                                                               ascending=False))

# 4 - Number of downloads of ggplot2
ggplot2 = downloads_RDD.filter(lambda x: x[6]=="ggplot2")
ggplot2_mapped = ggplot2.map(lambda x: (x[6], 1))
ggplot2_reduced = ggplot2_mapped.reduceByKey(lambda a,b: a+b)



package_RDD.foreachRDD(save_package_count)
package_RDD_reduced.foreachRDD(save_top_package)
country_RDD.foreachRDD(save_country)
ggplot2_reduced.foreachRDD(save_ggplot2_count)
#package_RDD.pprint()
#package_RDD_reduced.pprint(1)
#country_RDD.pprint(5)
#ggplot2_reduced.pprint()



In [3]:
ssc.start()
time.sleep(180)
ssc.stop(stopSparkContext=False)